In [3]:
import pandas as pd
import json
with open('data.json') as f:
    lines = f.readlines()
    
df = pd.DataFrame([json.loads(x) for x in lines])

In [4]:
N = 500
df = df.head(N)

In [5]:
df = df.drop('abstract', axis=1)

In [6]:
df.shape

(500, 13)

In [8]:
df.head(10)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"
5,0704.0006,Yue Hin Pong,Y. H. Pong and C. K. Law,Bosonic characters of atomic Cooper pairs acro...,"6 pages, 4 figures, accepted by PRA",None,10.1103/PhysRevA.75.043613,None,cond-mat.mes-hall,None,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2015-05-13,"[[Pong, Y. H., ], [Law, C. K., ]]"
6,0704.0007,Alejandro Corichi,"Alejandro Corichi, Tatjana Vukasinac and Jose ...",Polymer Quantum Mechanics and its Continuum Limit,"16 pages, no figures. Typos corrected to match...","Phys.Rev.D76:044016,2007",10.1103/PhysRevD.76.044016,IGPG-07/03-2,gr-qc,None,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-11-26,"[[Corichi, Alejandro, ], [Vukasinac, Tatjana, ..."
7,0704.0008,Damian Swift,Damian C. Swift,Numerical solution of shock and ramp compressi...,Minor corrections,"Journal of Applied Physics, vol 104, 073536 (2...",10.1063/1.2975338,"LA-UR-07-2051, LLNL-JRNL-410358",cond-mat.mtrl-sci,http://arxiv.org/licenses/nonexclusive-distrib...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2009-02-05,"[[Swift, Damian C., ]]"
8,0704.0009,Paul Harvey,"Paul Harvey, Bruno Merin, Tracy L. Huard, Luis...","The Spitzer c2d Survey of Large, Nearby, Inste...",None,"Astrophys.J.663:1149-1173,2007",10.1086/518646,None,astro-ph,None,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2010-03-18,"[[Harvey, Paul, ], [Merin, Bruno, ], [Huard, T..."
9,0704.0010,Sergei Ovchinnikov,Sergei Ovchinnikov,"Partial cubes: structures, characterizations, ...","36 pages, 17 figures",None,None,None,math.CO,None,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Ovchinnikov, Sergei, ]]"


In [13]:
json_data = df.to_json(orient = 'records')
df.to_json(r'dataframe.json', orient = 'records')
print(json_data)

[{"id":"0704.0001","submitter":"Pavel Nadolsky","authors":"C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan","title":"Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies","comments":"37 pages, 15 figures; published version","journal-ref":"Phys.Rev.D76:013009,2007","doi":"10.1103\/PhysRevD.76.013009","report-no":"ANL-HEP-PR-07-12","categories":"hep-ph","license":null,"versions":[{"version":"v1","created":"Mon, 2 Apr 2007 19:18:42 GMT"},{"version":"v2","created":"Tue, 24 Jul 2007 20:10:27 GMT"}],"update_date":"2008-11-26","authors_parsed":[["Bal\u00e1zs","C.",""],["Berger","E. L.",""],["Nadolsky","P. M.",""],["Yuan","C. -P.",""]]},{"id":"0704.0002","submitter":"Louis Theran","authors":"Ileana Streinu and Louis Theran","title":"Sparsity-certifying Graph Decompositions","comments":"To appear in Graphs and Combinatorics","journal-ref":null,"doi":null,"report-no":null,"categories":"math.CO cs.CG","license":"http:\/\/arxiv.org\/licenses\/nonexclusive

In [7]:
# from urllib.request import urlopen

# myKey = "WX923RkrwWYQE4UGSt4GHEFk7EZYgwUJ5adt"
# url = "https://gender-api.com/get?key=" + myKey + "&name="

# for index, row in df.head(100).iterrows():
#     firstName = row["submitter"].split(' ')[0] 
    
#     response = urlopen(url+firstName)
#     decoded = response.read().decode('utf-8')
#     data = json.loads(decoded)
#     df.at[index,'gender'] = data["gender"]
 